In [1]:
import sys
username = "alisa"
sys.path.append(f"/home/{username}/git/AstroDeepLearning/")
import torch
from ADL.model.MDN_Regression_torch import MDN_Regression, DeepEnsemble_MDN
from ADL.dataset.Planck_torch import Planck_Regression_Dataset, StratifiedSampler
from torch.utils.data import DataLoader
from ADL.model import pixels
import numpy as np
from matplotlib import pyplot as plt

## Обучение ансамбля моделей сегментации

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
optimizer = torch.optim.Adam
scheduler = torch.optim.lr_scheduler.ExponentialLR
ens = DeepEnsemble_MDN(MDN_Regression, 
    {'sizes': [512, 256, 128, 1], 'p': 0.0},
    n_models=5, device=device, model_save_path=f"/home/{username}/Models/Planck_MDN/5model_sampler/",
    
    optimizer=optimizer, 
    optimizer_args={'lr': 0.001, 'weight_decay': 0.001},
    scheduler=scheduler,
    scheduler_args={'gamma': 0.95},
                      )
device

'cuda:0'

In [3]:
dataset_train = Planck_Regression_Dataset(f"/home/{username}/Data/Planck/healpix_nside2/",
                               f"/home/{username}/Data/Planck/cluster_cats/with_coords/PSZ2_z.csv", 
                                    pix2=pixels.default["train"],
                                         patch_size=32)
dataset_val = Planck_Regression_Dataset(f"/home/{username}/Data/Planck/healpix_nside2/",
                               f"/home/{username}/Data/Planck/cluster_cats/with_coords/PSZ2_z.csv", 
                                    pix2=pixels.default["val"],
                                        patch_size=32)
dataset_test = Planck_Regression_Dataset(f"/home/{username}/Data/Planck/healpix_nside2/",
                               f"/home/{username}/Data/Planck/cluster_cats/with_coords/PSZ2_z.csv", 
                                    pix2=pixels.default["test"],
                                        patch_size=32)
batch_size = 128
sampler_train = StratifiedSampler(dataset_train, batch_size=batch_size, n_batches=90)
sampler_val = StratifiedSampler(dataset_val, batch_size=batch_size, n_batches=10)
sampler_test = StratifiedSampler(dataset_test, batch_size=batch_size, n_batches=10)

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False, 
                              sampler=sampler_train)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False,
                               sampler=sampler_val)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False,
                                sampler=sampler_test)

In [4]:
def sigma_nmad(true, preds):
    diff = preds - true
    m = np.median(diff)
    return 1.48 * np.median(np.abs((diff - m) / (1 + true)))

In [ ]:
ens.fit({"train": dataloader_train, "val": dataloader_val, "test": dataloader_test},
    epochs=100, 
    verbose=True,    
    metrics={'S_nmad': sigma_nmad})